In [1]:
from multiprocessing import Pool, Process, Queue
from datetime import datetime, timedelta
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

In [2]:
# генерация random user agents для обхода блокировок от сайта
!pip install random_user_agent
import random
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   
user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=300)
user_agents = user_agent_rotator.get_user_agents()
user_agent = user_agent_rotator.get_random_user_agent()

agents_list = []
for i in range(150):
    i = user_agent_rotator.get_random_user_agent()
    agents_list.append(i)

     |████████████████████████████████| 8.2MB 4.0MB/s 


In [20]:
# сбор ссылок по страницам
%%time
range_n = [i for i in range(0, 5944, 21)]


def fill_urls(number):
    links = []
    http = 'http://kyym.ru/sonunnar?start={}'.format(number)
    page = requests.get(http, headers = {'User-Agent': random.choice(agents_list)})
    soup = bs(page.text, 'html.parser')
    news_page = soup.find_all('h2')
    for news in news_page:
        address = 'http://kyym.ru'+news.find('a')['href']
        links.append(address)
    return links


big_links = []
p = Pool(24)


for links in p.map(fill_urls, range_n):
    big_links.extend(links)
print('number of news:', len(big_links))

p.terminate()
p.join()

number of news: 5958
CPU times: user 290 ms, sys: 459 ms, total: 749 ms
Wall time: 24.3 s


In [21]:
df = pd.DataFrame(big_links, columns=['link'])
df.to_csv('kyym-links.csv', index=False)

In [24]:
# запуск сбора данных из ссылок
%%time


def parse(link):
    dataframe = []
    page = requests.get(link, headers = {'User-Agent': random.choice(agents_list)})
    if page.status_code==200:
        soup = bs(page.text, 'html.parser')
        if soup is not None:
            if soup.find('title') is not None:
                title = soup.find('title').text
                if soup.find('time') is not None:
                    date = soup.find('time').text.split(' ')[4]
                    if soup.find('section', class_='article-content clearfix') is not None:
                        all_text = soup.find('section', class_='article-content clearfix').text
                        all_text = re.sub(r"\s{2,}", "", all_text).replace('\n', '')
                        dataframe.append((date, title, link, all_text))
                        return dataframe


df_info = []
p = Pool(24)

for dataframe in p.map(parse, big_links):
    if dataframe:
        df_info.extend(dataframe)
    else: 
        continue
p.terminate()
p.join()

len(df_info)

CPU times: user 2.91 s, sys: 2.78 s, total: 5.69 s
Wall time: 3min 59s


In [23]:
df = pd.DataFrame(df_info, columns=['date', 'title', 'link', 'all_text'])
df.to_csv('kyym-novosti.csv', index=False)